In [ ]:
import math
import random

GRID_SIZE = 10
SHIP_HEALTH = {"flagship": 10, "support": 3}
DAMAGE_PER_HIT = 2

ships = {
    "A1*": {"type": "flagship", "health": SHIP_HEALTH["flagship"], "player": "A"},
    "A2": {"type": "support", "health": SHIP_HEALTH["support"], "player": "A"},
    "A3": {"type": "support", "health": SHIP_HEALTH["support"], "player": "A"},
    "A4": {"type": "support", "health": SHIP_HEALTH["support"], "player": "A"},
    "B1*": {"type": "flagship", "health": SHIP_HEALTH["flagship"], "player": "B"},
    "B2": {"type": "support", "health": SHIP_HEALTH["support"], "player": "B"},
    "B3": {"type": "support", "health": SHIP_HEALTH["support"], "player": "B"},
    "B4": {"type": "support", "health": SHIP_HEALTH["support"], "player": "B"},
}

# Initialize the grid with predefined positions for ships
def create_grid():
    grid = [["-" for _ in range(GRID_SIZE)] for _ in range(GRID_SIZE)]
    grid[0][0], grid[0][1], grid[1][0], grid[1][1] = "A1*", "A2", "A3", "A4"  # Player A ships
    grid[9][9], grid[9][8], grid[8][9], grid[8][8] = "B1*", "B2", "B3", "B4"  # Player B ships
    return grid

# Function to print the grid in the required format
def display_grid(grid):
    for row in grid:
        print(" ".join(row))
    print("\n")

# Function to handle ship movement
def move_ship(grid, ship, current_pos, new_pos):
    x, y = current_pos
    new_x, new_y = new_pos
    if 0 <= new_x < GRID_SIZE and 0 <= new_y < GRID_SIZE and grid[new_x][new_y] == "-":
        grid[new_x][new_y], grid[x][y] = grid[x][y], "-"
        return True
    return False

# Function to perform an attack on a given position
def attack(grid, attacker, target_pos):
    tx, ty = target_pos
    target_ship = grid[tx][ty]
    if target_ship in ships:
        ships[target_ship]["health"] -= DAMAGE_PER_HIT
        if ships[target_ship]["health"] <= 0:
            grid[tx][ty] = "-"
            print(f"{target_ship} destroyed!")
        return True
    return False

# Check if the game is over (if any flagship is destroyed)
def game_over():
    if ships["A1*"]["health"] <= 0:
        print("Player B wins!")
        return True
    elif ships["B1*"]["health"] <= 0:
        print("Player A wins!")
        return True
    return False

# Generate possible moves for a player
def get_moves(grid, player):
    moves = []
    for i in range(GRID_SIZE):
        for j in range(GRID_SIZE):
            if grid[i][j] in ships and ships[grid[i][j]]["player"] == player:
                ship = grid[i][j]
                for dx in [-1, 0, 1]:
                    for dy in [-1, 0, 1]:
                        new_x, new_y = i + dx, j + dy
                        if 0 <= new_x < GRID_SIZE and 0 <= new_y < GRID_SIZE:
                            if grid[new_x][new_y] == "-":
                                moves.append(("move", ship, (i, j), (new_x, new_y)))
                            elif grid[new_x][new_y] in ships and ships[grid[new_x][new_y]]["player"] != player:
                                moves.append(("attack", ship, (i, j), (new_x, new_y)))
    return moves

# Apply a move (either move or attack) to the grid
def apply_move(grid, move):
    move_type, ship, from_pos, to_pos = move
    if move_type == "move":
        move_ship(grid, ship, from_pos, to_pos)
    elif move_type == "attack":
        attack(grid, ship, to_pos)

# Minimax with Alpha-Beta Pruning for AI decision-making
def minimax(grid, depth, alpha, beta, maximizing_player):
    if depth == 0 or game_over():
        return evaluate_board()

    moves = get_moves(grid, "B" if maximizing_player else "A")
    if maximizing_player:
        max_eval = -math.inf
        for move in moves:
            new_grid = [row[:] for row in grid]
            apply_move(new_grid, move)
            eval = minimax(new_grid, depth - 1, alpha, beta, False)
            max_eval = max(max_eval, eval)
            alpha = max(alpha, eval)
            if beta <= alpha:
                break
        return max_eval
    else:
        min_eval = math.inf
        for move in moves:
            new_grid = [row[:] for row in grid]
            apply_move(new_grid, move)
            eval = minimax(new_grid, depth - 1, alpha, beta, True)
            min_eval = min(min_eval, eval)
            beta = min(beta, eval)
            if beta <= alpha:
                break
        return min_eval

# ExpectiMiniMax for AI decision-making considering probabilities
def expectiminimax(grid, depth):
    if depth == 0 or game_over():
        return evaluate_board()

    moves = get_moves(grid, "B")
    total_eval = 0
    probability = 1 / len(moves) if moves else 1  # Equal probability for each move

    for move in moves:
        new_grid = [row[:] for row in grid]
        apply_move(new_grid, move)
        eval = expectiminimax(new_grid, depth - 1)
        total_eval += eval * probability
    
    return total_eval

# Function to evaluate the board state
def evaluate_board():
    a_health = ships["A1*"]["health"]
    b_health = ships["B1*"]["health"]
    return b_health - a_health  # Positive if B is favored, negative if A is favored

# Function to get user input for moves and attacks
def get_user_move(grid):
    while True:
        ship = input("Enter your ship (A1*, A2, A3, or A4): ")
        if ship not in ships or ships[ship]["player"] != "A":
            print("Invalid ship. Try again.")
            continue

        action = input("Enter 'move' to move or 'attack' to attack: ").lower()
        x, y = map(int, input("Enter target coordinates (x y): ").split())

        if action == "move":
            if move_ship(grid, ship, find_position(grid, ship), (x, y)):
                return
            else:
                print("Invalid move. Try again.")
        elif action == "attack":
            if attack(grid, ship, (x, y)):
                return
            else:
                print("Invalid attack. Try again.")

# Find the position of a ship on the grid
def find_position(grid, ship):
    for i in range(GRID_SIZE):
        for j in range(GRID_SIZE):
            if grid[i][j] == ship:
                return (i, j)
    return None

# Main game loop
def play_game():
    grid = create_grid()
    display_grid(grid)

    while not game_over():
        print("Player A's turn:")
        get_user_move(grid)
        display_grid(grid)
        if game_over():
            break

        print("Player B's (AI) turn:")
        best_move = None
        best_score = -math.inf
        
        # Choose between Minimax and ExpectiMiniMax based on the complexity desired
        use_expectiminimax = True  # Change this to False to use Minimax
        
        if use_expectiminimax:
            best_score = expectiminimax(grid, 3)  # Using ExpectiMiniMax
            # Here you can implement move selection based on best_score if needed
        else:
            for move in get_moves(grid, "B"):
                new_grid = [row[:] for row in grid]
                apply_move(new_grid, move)
                score = minimax(new_grid, 3, -math.inf, math.inf, False)
                if score > best_score:
                    best_score = score
                    best_move = move
            apply_move(grid, best_move)  # Apply the best move found by Minimax

        display_grid(grid)
        if game_over():
            break

# Run the game
play_game()


A1* A2 - - - - - - - -
A3 A4 - - - - - - - -
- - - - - - - - - -
- - - - - - - - - -
- - - - - - - - - -
- - - - - - - - - -
- - - - - - - - - -
- - - - - - - - - -
- - - - - - - - B4 B3
- - - - - - - - B2 B1*


Player A's turn:
- A2 - - - - - - - -
A3 A4 - - - - - - - -
- - - A1* - - - - - -
- - - - - - - - - -
- - - - - - - - - -
- - - - - - - - - -
- - - - - - - - - -
- - - - - - - - - -
- - - - - - - - B4 B3
- - - - - - - - B2 B1*


Player B's (AI) turn:
- A2 - - - - - - - -
A3 A4 - - - - - - - -
- - - A1* - - - - - -
- - - - - - - - - -
- - - - - - - - - -
- - - - - - - - - -
- - - - - - - - - -
- - - - - - - - - -
- - - - - - - - B4 B3
- - - - - - - - B2 B1*


Player A's turn:
Invalid attack. Try again.
